In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malayalam-news/HackFake Database for the Hackathon - Sheet1.csv


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

In [3]:
#Read dataset
FILEPATH='/kaggle/input/malayalam-news'
df_train=pd.read_csv(os.path.join(FILEPATH,'HackFake Database for the Hackathon - Sheet1.csv'))[:500]
df_train = df_train.drop(['Website (source)', 'Timeline/Date','Title'], axis=1)[2:].reset_index(drop=True)
df_train=df_train.dropna(axis=0)

In [4]:
df_train.columns

Index(['Heading of News Article', 'Text of News Article', 'Hate speech',
       'Misleading', 'Disinformation', 'Rumor/Hoax', 'Sensationalism',
       'Credible'],
      dtype='object')

In [5]:
X=df_train[['Text of News Article']]
#'Heading of News Article'
y=df_train[['Hate speech','Misleading','Disinformation','Rumor/Hoax','Sensationalism']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train=np.array(y_train).astype('int')
y_test=np.array(y_test).astype('int')

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout

#Custom CallBack and Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=X.shape[0]*20,
  decay_rate=1,
  staircase=False)


def tf_model():
    # Input layer
    input_layer = Input(shape=(), dtype=tf.string, name='text_input')

    # Preprocessor layer
    preprocessor_layer = hub.KerasLayer(
        "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
    )
    preprocessed_text = preprocessor_layer(input_layer)

    # Encoder layer
    encoder_layer = hub.KerasLayer(
        "https://tfhub.dev/google/LaBSE/2",
        trainable=False
    )

    embedded_text = encoder_layer(preprocessed_text)["sequence_output"]

    # Pass the embeddings through LSTM layer
    lstm_layer = LSTM(units=64, return_sequences=True)(embedded_text)

    # Bidirectional layer
    bi_dir_layer = Bidirectional(LSTM(64, return_sequences=False))(lstm_layer)

    # Batch normalization
    norm_layer = BatchNormalization()(bi_dir_layer)

    # Dense layer
    den_layer = Dense(units=32, activation='relu')(norm_layer)

    # Dropout layer
    drop_layer = Dropout(0.2)(den_layer)

    # Dense layer
    penum_layer = Dense(units=16, activation='relu')(drop_layer)

    # Output layer
    output_layer = Dense(units=5, activation='sigmoid', name='output')(penum_layer)

    # Create the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

# Create the model
model = tf_model()

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['text_input[0][0]']          
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [7]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['acc']
)

In [8]:
model.fit(X_train,y_train,epochs=30,callbacks=[early_stopping],validation_data=[X_test,y_test],batch_size=200)

Epoch 1/30
2/2 [==============================] - 159s 87s/step - loss: 0.7446 - acc: 0.1024 - val_loss: 0.6826 - val_acc: 0.1879
Epoch 2/30
2/2 [==============================] - 136s 82s/step - loss: 0.6493 - acc: 0.1386 - val_loss: 0.6757 - val_acc: 0.2788
Epoch 3/30
2/2 [==============================] - 160s 106s/step - loss: 0.5996 - acc: 0.2018 - val_loss: 0.6730 - val_acc: 0.3333
Epoch 4/30
2/2 [==============================] - 139s 84s/step - loss: 0.5637 - acc: 0.2530 - val_loss: 0.6693 - val_acc: 0.3273
Epoch 5/30
2/2 [==============================] - 176s 121s/step - loss: 0.5219 - acc: 0.2470 - val_loss: 0.6663 - val_acc: 0.3939
Epoch 6/30
2/2 [==============================] - 138s 84s/step - loss: 0.4829 - acc: 0.2892 - val_loss: 0.6643 - val_acc: 0.4606
Epoch 7/30
2/2 [==============================] - 163s 107s/step - loss: 0.4600 - acc: 0.3223 - val_loss: 0.6631 - val_acc: 0.4667
Epoch 8/30
2/2 [==============================] - 173s 118s/step - loss: 0.4348 - acc: 

In [9]:
model.predict(X_train.iloc[5])

1/1 [==============================] - 3s 3s/step


array([[0.5403876 , 0.30185243, 0.31478047, 0.2511711 , 0.35709497]],
      dtype=float32)

In [10]:
y_train[5]

array([1, 0, 0, 0, 0])

In [11]:
model.predict(X_test.iloc[10])

1/1 [==============================] - 0s 379ms/step


array([[0.5790241 , 0.61295384, 0.5748369 , 0.50388145, 0.63605136]],
      dtype=float32)

In [12]:
y_test[10]

array([0, 0, 0, 0, 1])

In [13]:
model.predict([''])

1/1 [==============================] - 3s 3s/step


array([[0.48189184, 0.35262322, 0.35760635, 0.39276406, 0.42054167]],
      dtype=float32)

In [14]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))